In [ ]:
import cv2
import numpy as np
import math

# meta data
##############################################
# image name
img_name = "B6216184 Mr.Rachapon Pongkittisak"

# ROI of raw data
start_x = 528#189
start_y  = 319#183
stop_x = 1367#860
stop_y= 903#650

# number of image in row and col
row = 9
col =16

# off set
x_start_offset = 3
x_stop_offset = 1
y_start_offset = 3
y_stop_offset = 1

alpha = 0.4
##############################################

# match selected img on roi image and display result
def click_event(event, x, y, flafs, param):
    if(event == cv2.EVENT_LBUTTONDOWN):
        global alpha, img_roi, max_x_img, max_y_img
#         print(x, y)
        x_index = math.ceil(x/max_x_img)
        y_index = math.ceil(y/max_y_img)
#         print(x_index, y_index)
        
        x_to = int(x_index * max_x_img) - x_stop_offset
        x_from = int(x_to - max_x_img) + x_start_offset
#         print("start_x: ",x_from ,",stop_x: ",x_to)
        
        y_to = int(y_index * max_y_img) - y_stop_offset
        y_from = int(y_to - max_y_img) + y_start_offset
#         print("start_y: ", y_from, ",stop_y: ", y_to)
        
        selected_roi = img_roi[y_from:y_to, x_from:x_to]
        cv2.imwrite("./image/selected_img.jpg", selected_roi)
        img_selected_roi = cv2.imread("./image/selected_img.jpg")
        cv2.imshow("Selected Image", img_selected_roi)
        
        gray = cv2.cvtColor(img_roi, cv2.COLOR_BGR2GRAY)
        template = cv2.imread("./image/selected_img.jpg", 0)
        w, h = template.shape[::-1]

        res = cv2.matchTemplate(gray, template, cv2.TM_CCOEFF_NORMED)
        threshold = 0.8
        loc = np.where(res >= threshold)

        display_img = img_roi.copy()
        for pt in zip(*loc[::-1]):
            cv2.rectangle(display_img, pt, (pt[0] + w, pt[1] + h), (0, 0, 255), -1) 
        new_img = cv2.addWeighted(display_img, alpha, img_roi, 1 - alpha, 0)    
        cv2.imshow(img_name+"-Result", new_img)

#select ROI
img = cv2.imread("./image/Data01.png")
# img2 = cv2.imread("./image/Data01.png")
# cv2.imshow(img_name+"-Original", img)
# cv2.imshow(img_name+"-Original2", img2)

roi = img[start_y:stop_y, start_x:stop_x]
cv2.imwrite("./image/Data_ROI.jpg", roi)
img_roi = cv2.imread("./image/Data_ROI.jpg")
cv2.imshow(img_name+"-ROI", img_roi)

#select ROI specified at one needed img
# print(img_roi.shape[0], img_roi.shape[1])
max_x_img = img_roi.shape[1]/col
max_y_img = img_roi.shape[0]/row
# print(max_x_img)
# print(max_y_img)
cv2.setMouseCallback(img_name+"-ROI", click_event)

cv2.waitKey(0)
cv2.destroyAllWindows()

